In [1]:
import glob
import dask
import pandas as pd

In [7]:
orderbook_list = sorted(glob.glob('datasets/btcusdt/orderbook/*.csv.gz'))
# quote_list = sorted(glob.glob('datasets/btcusdt/quotes/*.csv.gz'))

In [13]:
orderbook_list = sorted(glob.glob('/Users/mac/Desktop/FBD_Project/datasets/btcusdt/orderbook/*.csv.gz'))
quote_list = sorted(glob.glob('/Users/mac/Desktop/FBD_Project/datasets/btcusdt/quotes/*.csv.gz'))

In [27]:
df_ob = pd.read_csv(orderbook_list[0]).drop(columns=['exchange', 'local_timestamp'])
df_ob.shape

(2049046, 22)

In [30]:
df_quote = pd.read_csv(quote_list[0], usecols=['timestamp', 'ask_amount', 'ask_price', 'bid_price', 'bid_amount'])

# calculate mid price and bidask spread
df_quote['mid_price'] = df_quote[['ask_price', 'bid_price']].mean(axis=1)
df_quote['ba_spread'] = (df_quote['ask_price'] - df_quote['bid_price'])
df_quote['imbalance'] = df_quote['bid_amount']/(df_quote['bid_amount'] + df_quote['ask_amount'])


In [32]:
df_quote.iloc[:30]

,timestamp,ask_amount,ask_price,bid_price,bid_amount,mid_price,ba_spread,imbalance
0,1662076800125000,15.416,20122.6,20122.5,4.548,20122.55,0.1,0.227810
1,1662076802747000,15.416,20122.6,20122.5,4.748,20122.55,0.1,0.235469
2,1662076803027000,15.538,20122.6,20122.5,4.748,20122.55,0.1,0.234053
3,1662076803307000,15.538,20122.6,20122.5,4.741,20122.55,0.1,0.233789
4,1662076803559000,15.466,20122.6,20122.5,0.193,20122.55,0.1,0.012325
5,1662076803573000,11.887,20122.6,20122.5,1.774,20122.55,0.1,0.129859
6,1662076803585000,11.872,20122.6,20122.5,1.674,20122.55,0.1,0.123579
7,1662076803595000,10.954,20122.6,20122.5,0.218,20122.55,0.1,0.019513
8,1662076803606000,11.893,20122.6,20122.5,0.203,20122.55,0.1,0.016782
9,1662076803613000,11.883,20122.6,20122.5,0.203,20122.55,0.1,0.016796
